# 问题：预测航班延误

本笔记本的多个目标为：
- 处理下载的 ZIP 文件并利用其创建数据集
- 执行探索性数据分析 (EDA)
- 建立基准模型
- 将简单模型转化为集成模型
- 进行超参数优化
- 确认特征重要性

## 业务场景简介
您在一家行程预订网站工作，该网站致力于改善航班延误的旅客的客户体验。该公司希望开发一种功能，让美国的乘客在预订国内最繁忙机场的到港或离港航班时，能够知道航班是否会因天气原因而延误。

您的任务是利用机器学习来确定航班是否会因天气原因而延误，从而在一定程度上解决这一问题。您可以访问大型航空公司国内航班的准点率数据集。您可以使用这一数据来训练机器学习模型，以便预测最繁忙机场的航班是否会延误。

## 关于该数据集
该数据集包含经过认证的美国航空公司报告的预定和实际的出发与到达时间。对于预定行程的国内旅客产生的收入，这些航空公司至少占据了 1%。数据由美国交通统计局 (BTS) 下属的航空公司信息办公室收集。数据集包含 2013 到 2018 年间航班的日期、时刻、出发地、目的地、航空公司、航程以及航班延误状态。

### 特征
有关数据集内的特征的更多信息，请参阅 [准点延误数据集特征](https://www.transtats.bts.gov/Fields.asp)。

### 数据集属性  
网站：https://www.transtats.bts.gov/

本实验使用的数据集由美国交通统计局 (BTS) 下属的航空公司信息办公室编制，取自航空公司准点率数据库，网站为 https://www.transtats.bts.gov/DatabaseInfo.asp?DB_ID=120&amp;DB_URL=Mode_ID=1&amp;Mode_Desc=Aviation&amp;Subject_ID2=0。

# 步骤 1：问题定义和数据收集

在开始这个项目时，先在下面用几句话来总结在这个场景中要解决的业务问题和要实现的业务目标。同时说明您想让自己的团队衡量的业务指标。确定这些信息后，请清楚地写出机器学习问题陈述。最后，添加一两条备注，说明问题陈述所对应的机器学习的类型。

#### <span style="color: blue;">项目演示：在项目演示中总结这些详细信息。</span>

### 1. 确定 ML 是否是适合部署的解决方案以及原因。

In [ ]:
# 在此处写下答案

### 2. 定义业务问题、成功指标和期望的 ML 输出。

In [ ]:
# 在此处写下答案

### 3. 确定您要解决的 ML 问题的类型。

In [ ]:
# 在此处写下答案

### 4. 分析您要使用的数据是否合适。

In [ ]:
# 在此处写下答案

### 设置

我们已经确定了工作重点，现在我们来进行一些设置，以便开始解决问题。

**注意：**该笔记本是在 `ml.m4.xlarge` 笔记本实例上创建和测试的。

In [ ]:
import os
from pathlib2 import Path
from zipfile import ZipFile
import time

import pandas as pd
import numpy as np
import subprocess

import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# 步骤 2：数据预处理和可视化  
在数据预处理阶段，您应该利用机会探索数据并将其可视化，以便更好地了解数据。首先，导入必要的库并将数据读入 Pandas dataframe。然后，探索数据。查看数据集的形状，探索要使用的列及其类型（数值型、分类型）。对各项特征执行基本统计，以便了解特征的均值和范围；深入分析目标列并确定其分布。

### 要考虑的具体问题
1. 您可以从对特征执行的基本统计中推断出什么？ 

2. 您可以从目标类的分布中推断出什么？

3. 您从数据探索中还推断出了什么？

#### <span style="color: blue;">项目演示：在项目演示中总结这些问题和其他类似问题的答案。</span>

首先将 Amazon S3 公有存储桶中的数据集引入这个笔记本环境。

In [ ]:
# 检查文件是否已位于所需路径中，或者是否需要下载

base_path = '/home/ec2-user/SageMaker/project/data/FlightDelays/'
csv_base_path = '/home/ec2-user/SageMaker/project/data/csvFlightDelays/'
file_path = 'On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2014_1.zip'

if not os.path.isfile(base_path + file_path):
    subprocess.run(['mkdir', '-p', base_path])
    subprocess.run(['mkdir', '-p', csv_base_path])
    subprocess.run(['aws', 's3', 'cp',
                    's3://aws-tc-largeobjects/ILT-TF-200-MLDWTS/flight_delay_project/csvFlightData-5/',
                    base_path,'--recursive'])
else:
    print('File already downloaded!')

In [ ]:
zip_files = [str(file) for file in list(Path(base_path).iterdir()) if '.zip' in str(file)]
len(zip_files)

#### 从 ZIP 文件中提取 CSV 文件

In [ ]:
def zip2csv(zipFile_name , file_path = '/home/ec2-user/SageMaker/project/data/csvFlightDelays'):
    """
    从 zip 文件中提取 csv 文件
    zipFile_name：zip 文件的名称
    file_path : 用于存储 csv 文件的文件夹名称
    """
    fname='On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2014_1.csv'
    try:
        if not os.path.isfile(csv_base_path + fname):
            with ZipFile(zipFile_name, 'r') as z: 
                print(f'Extracting {zipFile_name} ') 
                z.extractall(path=file_path) 
    except:
        print(f'zip2csv failed for {zipFile_name}')

for file in zip_files:
    zip2csv(file)

print("Files Extracted")

In [ ]:
csv_files = [str(file) for file in list(Path(csv_base_path).iterdir()) if '.csv' in str(file)]
len(csv_files)

在加载 CSV 文件前，请先阅读提取的文件夹中的 HTML 文件。该 HTML 文件介绍了相关背景以及有关数据集内包含的特征的更多信息。

In [ ]:
from IPython.display import IFrame, HTML

IFrame(src="./data/csvFlightDelays/readme.html", width=1000, height=600)

#### 加载示例 CSV

在合并所有 CSV 文件前，需要了解各个 CSV 文件中的数据。先使用 Pandas 读取 `On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2018_9.csv` 文件。您可以使用 Python 内置的 `read_csv` 函数（[文档](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)）。

In [ ]:
df_temp = pd.read_csv(<CODE> # **在此输入代码**

**问题**：打印数据集内的行数和列数，并打印列名。

**提示**：使用 `<dataframe>.shape` 函数可以查看 DataFrame 的行和列，使用 `<dataframe>.columns` 函数可以查看列名。

In [ ]:
df_shape = # **在此输入代码**
print(f'Rows and columns in one csv file is {df_shape}')

**问题**：打印数据集的前 10 行。 

**提示**：使用内置的 Pandas 函数 `head(x)` 可以打印前 `x` 行。

In [ ]:
# 在此处输入代码

**问题**：打印数据集里的所有列。使用 `<dataframe>.columns` 可以查看列名。

In [ ]:
print(f'The column names are :')
print('#########')
for col in <CODE>:# **在此处输入代码**
    print(col)

**问题**：打印数据集里包含 'Del' 一词的所有列。这可以让您了解包含延误数据的列有多少。

**提示**：您可以使用 Python 的列表推导功能来列出符合特定 `if` 语句条件的值。

例如：`[x for x in [1,2,3,4,5] if x > 2]`  

**提示**：您可以使用 `in` 关键字（[文档](https://www.w3schools.com/python/ref_keyword_in.asp)）来确认值是否位于列表中。

例如：`5 in [1,2,3,4,5]`

In [1]:
# 在此处输入代码

下列问题也可以帮助您了解有关数据集的更多信息。

**问题**   
1. 数据集有多少个行和列？   
2. 数据集里包含多少个年份？   
3. 数据集的日期范围是多少？   
4.数据集里包含哪些航空公司？   
5.数据集里包含哪些出发地和目的地机场？

In [ ]:
print("The #rows and #columns are ", <CODE> , " and ", <CODE>)
print("The years in this dataset are: ", <CODE>)
print("The months covered in this dataset are: ", <CODE>)
print("The date range for data is :" , min(<CODE>), " to ", max(<CODE>))
print("The airlines covered in this dataset are: ", list(<CODE>))
print("The Origin airports covered are: ", list(<CODE>))
print("The Destination airports covered are: ", list(<CODE>))

**问题**：出发地和目的地机场的总数分别是多少？

**提示**：您可以使用 Pandas 函数 `values_count`（[文档](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html)），通过 `Origin` 列和 `Dest` 列来确定每个机场对应的值。

In [ ]:
counts = pd.DataFrame({'Origin':<CODE>, 'Destination':<CODE>})
counts

**问题**：根据数据集里的航班数量，打印出前 15 个出发地和目的地机场。

**提示**：您可以使用 Pandas 函数 `sort_values`（[文档](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html)）。

In [ ]:
counts.sort_values(by=<CODE>,ascending=False).head(15 )# 在此处输入代码

**问题**：掌握了有关某个航班的所有信息后，您能否预测其是否会延误？

In [ ]:
# 在此处输入答案：

假设您要从旧金山到洛杉矶出差。您想要通过一组特征来预测航班是否会延误，以便更好地预订洛杉矶的酒店。在登机之前，您会在这个数据集里了解多少特征？

`DepDelay`、`ArrDelay`、`CarrierDelay`、`WeatherDelay`、`NASDelay`、`SecurityDelay`、`LateAircraftDelay` 和 `DivArrDelay` 等列包含有关延误的信息。但是，这种延误可能发生在出发地，也可能发生在目的地。如果航班因天气骤变而导致延误了 10 分钟才到达，则这种数据对您预订洛杉矶的酒店没有帮助。

因此，要简化问题陈述，请考虑用以下各列来预测到达延误：<br>

`Year`、`Quarter`、`Month`、`DayofMonth`、`DayOfWeek`、`FlightDate`、`Reporting_Airline`、`Origin`、`OriginState`、`Dest`、`DestState`、`CRSDepTime`、`DepDelayMinutes`、`DepartureDelayGroups`、`Cancelled`、`Diverted`、`Distance`、`DistanceGroup`、`ArrDelay`、`ArrDelayMinutes`、`ArrDel15` 和 `AirTime`

您也可以将出发地和目的地机场筛选为：
- 热门机场：ATL、ORD、DFW、DEN、CLT、LAX、IAH、PHX、SFO
- 前 5 大航空公司：UA、OO、WN、AA、DL

这有助于减少要合并的 CSV 文件中的数据大小。

#### 合并所有 CSV 文件

**提示**：  
首先，创建一个空 DataFrame，用于复制每个文件中的 DataFrame。然后，针对 `csv_files` 列表中的每个文件执行以下操作：

1. 将 CSV 文件读入 DataFrame  
2. 根据 `filter_cols` 变量来筛选列

```
        columns = ['col1', 'col2']
        df_filter = df[columns]
```

3. 仅保留每个 subset_cols 中的 subset_vals。使用 Pandas 函数 `isin`（[文档](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isin.html)）来确认 `val` 是否位于 DataFrame 列中，然后选择包含它的行。

```
        df_eg[df_eg['col1'].isin('5')]
```

4. 将该 DataFrame 与空 DataFrame 合并 

In [ ]:
def combine_csv(csv_files, filter_cols, subset_cols, subset_vals, file_name = 'data/combined_files.csv'):
    """
    将 csv 文件合并到一个 DataFrame 中
    csv_files：csv 文件路径的列表
    filter_cols：要筛选的列的列表
    subset_cols：包含子集行的列的列表
    subset_vals：包含子集行的值的列表
    """
    # 创建一个空 DataFrame
    df = # 在此处输入代码 
    
    for file in csv_files:
        # 将 CSV 文件读入 DataFrame
        df_temp = pd.read_csv(<CODE>)# 在此输入代码
        
        # 根据 filter_cols 变量来筛选列
        # 例如 columns = ['col1', 'col2']
        # df_filter = df[columns]
        df_temp = # 在此处输入代码
        
        # 仅保留每个 subset_cols 中的 subset_vals
        ＃ 提示：使用 `isin` 函数来确认 val 是否位于 DataFrame 列中
        # 然后选择包含它的行
        # 例如 df[df['col1'].isin('5')]
        for col, val in zip(subset_cols,subset_vals):
            df_temp = # 在此处输入代码     
        
        ＃ 使用 Pandas 连结 `pd.concat`，以连结主 DataFrame 与每个文件的 DataFrame
        df = pd.concat([df, df_temp], axis=0)
    
        
    df.to_csv(file_name, index=False)
    print(f'Combined csv stored at {file_name}')

In [ ]:
#cols 是用于预测到达延误的列的列表 
cols = ['Year','Quarter','Month','DayofMonth','DayOfWeek','FlightDate',
        'Reporting_Airline','Origin','OriginState','Dest','DestState',
        'CRSDepTime','Cancelled','Diverted','Distance','DistanceGroup',
        'ArrDelay','ArrDelayMinutes','ArrDel15','AirTime']

subset_cols = ['Origin', 'Dest', 'Reporting_Airline']

# subset_vals 是热门出发地和目的地以及前 5 大航空公司的列表集合
subset_vals = [['ATL', 'ORD', 'DFW', 'DEN', 'CLT', 'LAX', 'IAH', 'PHX', 'SFO'],
               ['ATL', 'ORD', 'DFW', 'DEN', 'CLT', 'LAX', 'IAH', 'PHX', 'SFO'],
               ['UA', 'OO', 'WN', 'AA', 'DL']]

使用上述函数将所有不同文件合并到一个可以轻松读取的文件中。

**注意**：这一过程需要 5 到 7 分钟才能完成。

In [ ]:
start = time.time()
combine_csv(csv_files, cols, subset_cols, subset_vals)
print(f'csv\'s merged in {round((time.time() - start)/60,2)} minutes')

#### 加载数据集

加载已合并的数据集。

In [ ]:
data = pd.read_csv(<CODE>)# 在此处输入代码，以阅读合并的 csv 文件。

打印前 5 个记录。

In [ ]:
# 在此处输入代码 

下列问题也可以帮助您了解有关数据集的更多信息。

**问题**   
1. 数据集有多少个行和列？   
2. 数据集里包含多少个年份？   
3. 数据集的日期范围是多少？   
4. 数据集里包含哪些航空公司？   
5.数据集里包含哪些出发地和目的地机场？

In [ ]:
print("The #rows and #columns are ", <CODE> , " and ", <CODE>)
print("The years in this dataset are: ", list(<CODE>))
print("The months covered in this dataset are: ", sorted(list(<CODE>)))
print("The date range for data is :" , min(<CODE>), " to ", max(<CODE>))
print("The airlines covered in this dataset are: ", list(<CODE>))
print("The Origin airports covered are: ", list(<CODE>))
print("The Destination airports covered are: ", list(<CODE>))

下面我们来定义**目标列：is_delay**（航班到达延误超过 15 分钟时，其值为 1，否则为 0）。使用 `rename` 方法将 `ArrDel15` 列重命名为 `is_delay`。

**提示**：您可以使用 Pandas 函数 `rename`（[文档](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html)）。

例如：
```
df.rename(columns={'col1':'column1'}, inplace=True)
```

In [ ]:
data.rename(columns=<CODE>, inplace=True) # 在此处输入代码

查找各列的空值。您可以使用 `isnull()` 函数（[文档](https://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.isnull.html)）。

**提示**：`isnull()` 可以检测出特定值是否为空值，并在其位置给出布尔值（True 或 False）。使用 `sum(axis=0)` 函数可以计算列的总数。

In [ ]:
# 在此处输入代码

在 1658130 个行中，有 22540 个行缺少航班延误详情和飞行时间，占 1.3%。您可以删除或替换这些行。文档没有提到有关缺少信息的行的任何信息。

**提示**：使用 `~` 运算符可以从 `isnull()` 的输出选择非空值。

例如：
```
null_eg = df_eg[~df_eg['column_name'].isnull()]
```

In [ ]:
### 删除空值列
data = # 在此处输入代码

将 CRSDepTime 转化为 24 小时制时间。

In [ ]:
data['DepHourofDay'] = # 在此处输入代码

## **ML 问题陈述**
- 掌握了一组特征后，您能否预测出某个航班是否会延误 15 分钟以上？
- 目标变量只会有 0 和 1 两个值，因此您可以使用分类算法。

在开始建模前，最好先查看特征的分布和相关性等信息。
- 这可以让您了解数据中的任何非线性/模式。
    - 线性模型：添加幂/指数/交互特征
    - 尝试非线性模型
- 数据不平衡 
    - 选择反映的模型效果不会有偏差的指标（准确率与 AUC）
    - 使用加权/自定义损失函数
- 缺少数据
    - 基于简单的统计数据进行替换：均值、中位数、众数（数值变量）和频繁类（分类变量）
    - 基于集群进行替换（用 KNN 预测列值）
    - 删除列

### 数据探索

#### 查看延误类和不延误类

**提示**：使用 `groupby` 图（[文档](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html)）和 `bar` 图（[文档](https://matplotlib.org/tutorials/introductory/pyplot.html)）来绘制频率和类的分布。

In [ ]:
(data.groupby(<CODE>).size()/len(data) ).plot(kind='bar')# 在此处输入代码
plt.ylabel('Frequency')
plt.title('Distribution of classes')
plt.show()

**问题**：您能从柱状图中推断出有关延误率和不延误率的什么信息？

In [ ]:
# 在此处输入答案：

**问题**： 

- 哪个月的延误次数最多？
- 一天之中什么时候的延误次数最多？
- 星期几的延误次数最多？
- 哪家航空公司的延误次数最多？
- 哪个出发地和目的地机场的延误次数最多？
- 航程是不是与延误有关的一项因素？

In [ ]:
viz_columns = ['Month', 'DepHourofDay', 'DayOfWeek', 'Reporting_Airline', 'Origin', 'Dest']
fig, axes = plt.subplots(3, 2, figsize=(20,20), squeeze=False)
# fig.autofmt_xdate(rotation=90)

for idx, column in enumerate(viz_columns):
    ax = axes[idx//2, idx%2]
    temp = data.groupby(column)['is_delay'].value_counts(normalize=True).rename('percentage').\
    mul(100).reset_index().sort_values(column)
    sns.barplot(x=column, y="percentage", hue="is_delay", data=temp, ax=ax)
    plt.ylabel('% delay/no-delay')
    

plt.show()

In [ ]:
sns.lmplot( x="is_delay", y="Distance", data=data, fit_reg=False, hue='is_delay', legend=False)
plt.legend(loc='center')
plt.xlabel('is_delay')
plt.ylabel('Distance')
plt.show()

In [ ]:
# 在此处输入答案

### 特征

查看所有列及其具体类型。

In [ ]:
data.columns

筛选出所需的列：
- Date 是多余的数据，因为已经有 Year、Quarter、Month、DayofMonth 和 DayOfWeek 可以描述日期。
- 使用 Origin 和 Dest 代码，而不是 OriginState 和 DestState。
- 因为您只是在对航班是否延误进行分类，所以不需要 TotalDelayMinutes、DepDelayMinutes 和 ArrDelayMinutes。

将 DepHourofDay 视为分类变量，因为它与目标没有数量关系。
- 如果必须对其进行独热编码，则会多出 23 列。
- 处理分类变量的其他方法包括进行哈希编码、进行正则化均值编码以及将值拆分到存储桶中等等。
- 在这里只需拆分到存储桶中即可。

**提示**：要将列类型更改为类别，请使用 `astype` 函数（[文档](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html)）。

In [ ]:
data_orig = data.copy()
data = data[[ 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
       'Reporting_Airline', 'Origin', 'Dest','Distance','DepHourofDay', 'is_delay']]
categorical_columns = ['Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
       'Reporting_Airline', 'Origin', 'Dest', 'DepHourofDay', 'is_delay']
for c in categorical_columns:
    data[c] = data[c].astype('category')# 在此处输入代码

要使用独热编码，请针对上述选择的分类数据列使用 Pandas 函数 `get_dummies`。然后可以使用 Pandas 函数 `concat` 将生成的这些特征合并到原始数据集中。要对分类变量进行编码，您也可以使用关键字 `drop_first=True` 来进行*虚拟编码*。有关虚拟编码的更多信息，请参阅 https://en.wikiversity.org/wiki/Dummy_variable_(statistics)。

例如：
```
pd.get_dummies(df[['column1','columns2']], drop_first=True)
```

In [ ]:
data_dummies = pd.get_dummies(<CODE>, drop_first=True) # 在此处输入代码
data = pd.concat([<CODE>, <CODE>], axis = 1)
categorical_columns.remove('is_delay')
data.drop(categorical_columns,axis=1, inplace=True)

确认数据集和新列的长度。

In [ ]:
# 在此处输入代码

In [ ]:
# 在此处输入代码

**示例答案：** 
```
Index(['Distance', 'is_delay', 'Quarter_2', 'Quarter_3', 'Quarter_4',
       'Month_2', 'Month_3', 'Month_4', 'Month_5', 'Month_6', 'Month_7',
       'Month_8', 'Month_9', 'Month_10', 'Month_11', 'Month_12',
       'DayofMonth_2', 'DayofMonth_3', 'DayofMonth_4', 'DayofMonth_5',
       'DayofMonth_6', 'DayofMonth_7', 'DayofMonth_8', 'DayofMonth_9',
       'DayofMonth_10', 'DayofMonth_11', 'DayofMonth_12', 'DayofMonth_13',
       'DayofMonth_14', 'DayofMonth_15', 'DayofMonth_16', 'DayofMonth_17',
       'DayofMonth_18', 'DayofMonth_19', 'DayofMonth_20', 'DayofMonth_21',
       'DayofMonth_22', 'DayofMonth_23', 'DayofMonth_24', 'DayofMonth_25',
       'DayofMonth_26', 'DayofMonth_27', 'DayofMonth_28', 'DayofMonth_29',
       'DayofMonth_30', 'DayofMonth_31', 'DayOfWeek_2', 'DayOfWeek_3',
       'DayOfWeek_4', 'DayOfWeek_5', 'DayOfWeek_6', 'DayOfWeek_7',
       'Reporting_Airline_DL', 'Reporting_Airline_OO', 'Reporting_Airline_UA',
       'Reporting_Airline_WN', 'Origin_CLT', 'Origin_DEN', 'Origin_DFW',
       'Origin_IAH', 'Origin_LAX', 'Origin_ORD', 'Origin_PHX', 'Origin_SFO',
       'Dest_CLT', 'Dest_DEN', 'Dest_DFW', 'Dest_IAH', 'Dest_LAX', 'Dest_ORD',
       'Dest_PHX', 'Dest_SFO'],
      dtype='object')
```

现在可以开始训练模型了。在拆分数据前，请将 `is_delay` 列重命名为 `target`。

**提示**：您可以使用 Pandas 函数 `rename`（[文档](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html)）。

In [ ]:
data.rename(columns = {<CODE>:<CODE>}, inplace=True )# 在此处输入代码

## <span style="color:red">实验 2 结束</span>

将项目文件保存到本地计算机。按照以下步骤进行操作：

1. 在页面顶部，单击 **File（文件）**菜单。

1. 选择 **Download as（另存为）**，然后单击 **Notebook (.ipynb)**。 

这会将当前的笔记本下载到计算机上默认的下载文件夹中。

# 步骤 3：模型训练和评估

在将数据集从 dataframe 转换为可供机器学习算法使用的格式之前，您必须完成一些预备步骤。对于 Amazon SageMaker，您需要采取以下步骤：

1. 使用 `sklearn.model_selection.train_test_split` 将数据拆分为 `train_data`、`validation_data` 和 `test_data`。 
2. 将数据集转换为适合 Amazon SageMaker 训练作业使用的文件格式。可以是 CSV 文件或记录 protobuf。有关更多信息，请参阅 [适用于训练的常见数据格式](https://docs.aws.amazon.com/sagemaker/latest/dg/cdf-training.html)。 
3. 将数据上传到 Amazon S3 存储桶中。如果您从未创建过存储桶，请参阅 [创建存储桶](https://docs.aws.amazon.com/AmazonS3/latest/gsg/CreatingABucket.html)。 

使用以下单元格完成这些步骤。根据需要插入和删除单元格。

#### <span style="color: blue;">项目陈述：请在项目陈述中记下您在这一阶段做出的关键决定。</span>

### 训练测试拆分

In [ ]:
from sklearn.model_selection import train_test_split

def create_training_sets(data):
    """
    将 DataFrame 转换为训练、验证和测试
    params:
        data：要拆分数据集的 DataFrame
    返回值：
        train_features：训练特征数据集
        test_features：测试特征数据集 
        train_labels：训练数据集的标签
        test_labels：测试数据集的标签
        val_features：验证特征数据集
        val_labels：验证数据集的标签
    """
    # 从 DataFrame 提取目标变量并将类型转换为 float32
    ys = np.array(<CODE>).astype("float32") # 在此处输入代码
    
    # 删除所有不需要的列，包括目标列
    drop_list = # 在此处输入代码
    
    # 删除 drop_list 中的列并将数据转换为 float32 类型的 NumPy 数组
    xs = np.array(data.drop(<CODE>, axis=1)).astype("float32")# 在此处输入代码
    
    np.random.seed(0)

    # 使用 sklearn 函数 train_test_split 按训练 80% 和测试 20% 的比例拆分数据集
    # 示例：train_test_split(x, y, test_size=0.3)
    train_features, test_features, train_labels, test_labels = # 在此处输入代码
    
    # 再次使用 sklearn 函数将测试数据集拆分为 50% 用作验证和 50% 用作测试
    val_features, test_features, val_labels, test_labels = # 在此处输入代码
    
    
    return train_features, test_features, train_labels, test_labels, val_features, val_labels

In [ ]:
# 使用函数来创建您的数据集
train_features, test_features, train_labels, test_labels, val_features, val_labels = create_training_sets(data)

print(f"Length of train_features is: {<CODE>}")
print(f"Length of train_labels is: {<CODE>}")
print(f"Length of val_features is: {<CODE>}")
print(f"Length of val_labels is: {<CODE>}")
print(f"Length of test_features is: {<CODE>}")
print(f"Length of test_labels is: {<CODE>}")

**示例答案**
```
train_features 的行列数为：(1308472, 71)
train_labels 的行列数为：(1308472,)
val_features 的行列数为：(163559, 71)
val_labels 的行列数为：(163559,)
test_features 的行列数为：(163559, 71)
test_labels 的行列数为：(163559,)
```

### 基准分类模型

In [ ]:
import sagemaker
from sagemaker.predictor import csv_serializer
from sagemaker.amazon.amazon_estimator import RecordSet
import boto3

num_classes = # 在此处输入代码

# 实例化线性学习器评估程序对象
classifier_estimator = sagemaker.LinearLearner(role=sagemaker.get_execution_role(),
                                               train_instance_count=<CODE>,
                                               train_instance_type=<CODE>,
                                               predictor_type=<CODE>,
                                              binary_classifier_model_selection_criteria=<CODE>)

### 示例代码
```
num_classes = len(pd.unique(train_labels))
classifier_estimator = sagemaker.LinearLearner(role=sagemaker.get_execution_role(),
                                              train_instance_count=1,
                                              train_instance_type='ml.m4.xlarge',
                                              predictor_type='binary_classifier',
                                             binary_classifier_model_selection_criteria = 'cross_entropy_loss')
                                              
```

线性学习器接受内容类型为 protobuf 或 CSV 的训练数据以及内容类型为 protobuf、CSV 或 JSON 的推理请求。训练数据有特征和 ground-truth 标签，而推理请求中的数据只有特征。

在生产管道中，建议将数据转换为 Amazon SageMaker protobuf 格式并将其存储在 Amazon S3 中。但是，为了快速启动并运行，AWS 能够在数据集小到足以装入本地内存时提供便捷的 `record_set` 方法来进行转换和上传。该方法接受您已有的 NumPy 数组，所以我们在这里就用它。`RecordSet` 对象将会跟踪数据的临时 Amazon S3 位置。使用 `estimator.record_set` 函数来创建训练、验证和测试记录。然后使用 `estimator.fit` 函数启动训练作业。

In [ ]:
### 创建训练、验证和测试记录
train_records = linear.record_set(<CODE>,<CODE>, channel='train')# 在此处输入代码
val_records = linear.record_set(<CODE>,<CODE>, channel='validation')# 在此处输入代码
test_records = linear.record_set(<CODE>,<CODE>, channel='test')# 在此处输入代码

下面，使用您上传的数据集训练模型。

### 示例代码
```
linear.fit([train_records,val_records,test_records])
```

In [ ]:
### 安装分类器
# 在此处输入代码

## 模型评估
在本节中，您将评估自己训练的模型。首先，使用 `estimator.deploy` 函数（`initial_instance_count= 1` 且 `instance_type= 'ml.m4.xlarge'`）在 Amazon SageMaker 上部署模型。

In [ ]:
### 部署用于批量预测的终端节点
classifier_predictor = classifier_estimator.deploy(initial_instance_count=<CODE>,
                                                   instance_type=<CODE>) # 在此处输入代码

当终端节点为 'InService' 时，请评估模型使用测试集的效果。请使用 `predict_batches` 函数利用测试集来预测指标。

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_fscore_support

def predict_batches(predictor, features, labels):
    """
    返回评估结果
    predictor：模型的预测器对象
    features：输入模型的特征
    label：Ground Truth 目标值
    """
    prediction_batches = [predictor.predict(batch) for batch in np.array_split(features, 100)]

    # 解析 protobuf 响应来提取预测标签
    extract_label = lambda x: x.label['predicted_label'].float32_tensor.values
    preds = np.concatenate([np.array([extract_label(x) for x in batch]) for batch in prediction_batches])
    preds = preds.reshape((-1,))

    # 计算准确率
    accuracy = (preds == labels).sum() / labels.shape[0]
    print(f'Accuracy: {accuracy}')
    
    auc = roc_auc_score(labels, preds)
    print(f'AUC : {auc}')
    
    precision, recall, f1_score, _ = precision_recall_fscore_support(labels, preds, average = 'binary')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1_score: {f1_score}')
    
    confusion_matrix = pd.crosstab(index=labels, columns=np.round(preds), rownames=['True'], colnames=['predictions']).astype(int)
    plt.figure(figsize = (5,5))
    sns.heatmap(confusion_matrix, annot=True, fmt='.2f', cmap="YlGnBu").set_title('Confusion Matrix') 

要查看混淆矩阵，在测试数据集上运行 `predict_batches` 函数。

In [ ]:
# 在此处输入代码

### 要考虑的关键问题：
1. 您的模型在测试集上的表现与训练集上的表现相比如何？ 您可以从这种比较中推断出什么？ 

2. 准确率、查准率和查全率等指标结果之间有没有明显差异？ 如果是，为什么您会看到这些差异？ 

3. 考虑到您的业务状况和目标，您此时最需要考虑的指标是什么？ 为什么？

4. 您认为最重要指标的结果是否足以满足您的业务需求？ 如果不能，那么您在下一次迭代中可能会更改哪些内容（在特征设计部分，下一步是什么）？ 

用下面的单元格来回答这些问题和其他问题。根据需要插入和删除单元格。

#### <span style="color: blue;">项目演示：在项目演示中记录这些问题的答案以及您可能会在本节中回答的其他类似问题的答案。在项目演示中记录关键的详细信息和您所做的关键决定。</span>


**问题**：您可以从混淆矩阵中总结出什么？


In [ ]:
# 在此处输入答案：

## <span style="color:red">实验 3 结束</span>

将项目文件保存到本地计算机。按照以下步骤进行操作：

1. 在页面顶部，单击 **File（文件）**菜单。

1. 选择 **Download as（另存为）**，然后单击 **Notebook (.ipynb)**。 

这会将当前的笔记本下载到计算机上默认的下载文件夹中。

# 迭代 II

# 步骤 4：特征设计

现在，您已经完成了一次训练和评估模型的迭代。鉴于模型第一次获得的结果可能不足以解决您的业务问题，您可以对数据进行哪些更改以改进模型的表现？

### 要考虑的关键问题：
1. 两个主要类（延误与不延误）的平衡可能会对模型效果有怎样的影响？
2. 是否有任何相关联的特征？
3. 在此阶段，您是否可以执行可能会对模型性能产生积极影响的特征约简技术？ 
4. 您能否想办法添加更多数据/数据集？
4. 执行一些特征设计之后，模型效果与第一次迭代相比如何？

使用下面的单元格执行您认为可以提高模型性能的特定特征设计技术（根据上述问题）。根据需要插入和删除单元格。

#### <span style="color: blue;">项目演示：在项目演示中记录您在本节中使用的主要决策和方法，以及在再次评估模型后获得的任何新性能指标。</span>

在开始之前，思考一下为什么查准率和查全率在 80% 左右，而准确率为 99%。

#### 添加更多特征

1. 节假日
2. 天气

2014 到 2018 年的节假日列表是已知数据，因此您可以创建指示变量 **is_holiday** 来标记这些节假日。
我们假设节假日的航班延误比平时更多。请添加一个涵盖 2014 到 2018 年的节假日的布尔变量 `is_holiday`。

In [ ]:
# 来源：http://www.calendarpedia.com/holidays/federal-holidays-2014.html

holidays_14 = ['2014-01-01', '2014-01-20', '2014-02-17', '2014-05-26', '2014-07-04', '2014-09-01', '2014-10-13', '2014-11-11', '2014-11-27', '2014-12-25' ] 
holidays_15 = ['2015-01-01', '2015-01-19', '2015-02-16', '2015-05-25', '2015-06-03', '2015-07-04', '2015-09-07', '2015-10-12', '2015-11-11', '2015-11-26', '2015-12-25'] 
holidays_16 = ['2016-01-01', '2016-01-18', '2016-02-15', '2016-05-30', '2016-07-04', '2016-09-05', '2016-10-10', '2016-11-11', '2016-11-24', '2016-12-25', '2016-12-26']
holidays_17 = ['2017-01-02', '2017-01-16', '2017-02-20', '2017-05-29' , '2017-07-04', '2017-09-04' ,'2017-10-09', '2017-11-10', '2017-11-23', '2017-12-25']
holidays_18 = ['2018-01-01', '2018-01-15', '2018-02-19', '2018-05-28' , '2018-07-04', '2018-09-03' ,'2018-10-08', '2018-11-12','2018-11-22', '2018-12-25']
# holidays_19 = ['2019-01-01', '2019-01-21', '2019-02-18', '2019-05-27' , '2019-07-04', '2019-09-02' ,'2019-10-14', '2019-11-11','2019-11-28', '2019-12-25']
holidays = holidays_14+ holidays_15+ holidays_16 + holidays_17+ holidays_18

### 为节假日添加指示变量
data_orig['is_holiday'] = # 在此处输入代码 

天气数据获取自 https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&amp;stations=USW00023174,USW00012960,USW00003017,USW00094846,USW00013874,USW00023234,USW00003927,USW00023183,USW00013881&amp;dataTypes=AWND,PRCP,SNOW,SNWD,TAVG,TMIN,TMAX&amp;startDate=2014-01-01&amp;endDate=2018-12-31。
<br>

该数据集包含多个城市的风速、降水量、降雪量和温度信息，按其机场代码列出。

**问题**：雨、雪或大风造成的恶劣天气是否会导致航班延误？ 我们来确认一下。

In [ ]:
%%bash

aws s3 cp s3://aws-tc-largeobjects/ILT-TF-200-MLDWTS/flight_delay_project/daily-summaries.csv /home/ec2-user/SageMaker/project/data/
#wget 'https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&amp;stations=USW00023174,USW00012960,USW00003017,USW00094846,USW00013874,USW00023234,USW00003927,USW00023183,USW00013881&amp;dataTypes=AWND,PRCP,SNOW,SNWD,TAVG,TMIN,TMAX&amp;startDate=2014-01-01&amp;endDate=2018-12-31' -O /home/ec2-user/SageMaker/project/data/daily-summaries.csv

将与机场代码对应的天气数据导入数据集中。使用下面的气象站和机场进行分析，并创建一个名为 `airport` 的新列，将气象站映射到机场名称。

In [ ]:
weather = pd.read_csv(<CODE>) # 在此处输入代码，以读取daily-summaries.csv' 文件
station = ['USW00023174','USW00012960','USW00003017','USW00094846',
           'USW00013874','USW00023234','USW00003927','USW00023183','USW00013881'] 
airports = ['LAX', 'IAH', 'DEN', 'ORD', 'ATL', 'SFO', 'DFW', 'PHX', 'CLT']

### 将气象站映射到机场代码
station_map = # 在此处输入代码 
weather['airport'] = # 在此处输入代码 

从 `DATE` 列中创建另一个名为 `MONTH` 的列。

In [ ]:
weather['MONTH'] = weather[<CODE>].apply(lambda x: x.split('-')[1])# 在此处输入代码 
weather.head()

### 示例输出
```
  STATION DATE AWND PRCP SNOW SNWD TAVG TMAX TMIN airport MONTH
0 USW00023174 2014-01-01 16 0 NaN NaN 131.0 178.0 78.0 LAX 01
1 USW00023174 2014-01-02 22 0 NaN NaN 159.0 256.0 100.0 LAX 01
2 USW00023174 2014-01-03 17 0 NaN NaN 140.0 178.0 83.0 LAX 01
3 USW00023174 2014-01-04 18 0 NaN NaN 136.0 183.0 100.0 LAX 01
4 USW00023174 2014-01-05 18 0 NaN NaN 151.0 244.0 83.0 LAX 01
```

使用 `fillna()` 来分析并处理 `SNOW` 和 `SNWD` 列缺少的值。使用 `isna()` 函数来检查所有列缺少的值。

In [ ]:
weather.SNOW.fillna(<CODE>, inplace=True)# 在此处输入代码
weather.SNWD.fillna(<CODE>, inplace=True)# 在此处输入代码
weather.isna().sum()

**问题**：打印 TAVG、TMAX 和 TMIN 缺少值的行的索引。

**提示**：使用 `isna()` 函数找出缺少的行，然后使用 idx 变量上的列表来获取索引。

In [ ]:
idx = np.array([i for i in range(len(weather))])
TAVG_idx = # 在此处输入代码 
TMAX_idx = # 在此处输入代码 
TMIN_idx = # 在此处输入代码 
TAVG_idx

### 示例输出

```
array([ 3956, 3957, 3958, 3959, 3960, 3961, 3962, 3963, 3964,
        3965, 3966, 3967, 3968, 3969, 3970, 3971, 3972, 3973,
        3974, 3975, 3976, 3977, 3978, 3979, 3980, 3981, 3982,
        3983, 3984, 3985, 4017, 4018, 4019, 4020, 4021, 4022,
        4023, 4024, 4025, 4026, 4027, 4028, 4029, 4030, 4031,
        4032, 4033, 4034, 4035, 4036, 4037, 4038, 4039, 4040,
        4041, 4042, 4043, 4044, 4045, 4046, 4047, 13420])
```

您可以用特定气象站/机场的平均值来替换缺少的 TAVG、TMAX 和 TMIN 值。由于 TAVG_idx 缺少连续的行，因此无法用上个值进行替换。请使用均值进行替换。使用 `groupby` 函数来聚合采用均值的变量。

In [ ]:
weather_impute = weather.groupby([<CODE>]).agg({'TAVG':'mean','TMAX':'mean', 'TMIN':'mean' }).reset_index()# 在此处输入代码
weather_impute.head(2)

合并均值数据与天气数据。

In [ ]:
### 获得昨天的数据
weather = pd.merge(weather, weather_impute, how='left', left_on=['MONTH','STATION'], right_on = ['MONTH','STATION'])\
.rename(columns = {'TAVG_y':'TAVG_AVG',
                   'TMAX_y':'TMAX_AVG',
                   'TMIN_y':'TMIN_AVG',
                   'TAVG_x':'TAVG',
                   'TMAX_x':'TMAX',
                   'TMIN_x':'TMIN'})

再次检查缺少的值。

In [ ]:
weather.TAVG[TAVG_idx] = weather.TAVG_AVG[TAVG_idx]
weather.TMAX[TMAX_idx] = weather.TMAX_AVG[TMAX_idx]
weather.TMIN[TMIN_idx] = weather.TMIN_AVG[TMIN_idx]
weather.isna().sum()

从数据集里删除 `STATION,MONTH,TAVG_AVG,TMAX_AVG,TMIN_AVG,TMAX,TMIN,SNWD`

In [ ]:
weather.drop(columns=['STATION','MONTH','TAVG_AVG', 'TMAX_AVG', 'TMIN_AVG', 'TMAX' ,'TMIN', 'SNWD'],inplace=True)

将出发地和目的地天气状况添加到数据集内。

In [ ]:
### 添加起点站天气状况
data_orig = pd.merge(data_orig, weather, how='left', left_on=['FlightDate','Origin'], right_on = ['DATE','airport'])\
.rename(columns = {'AWND':'AWND_O','PRCP':'PRCP_O', 'TAVG':'TAVG_O', 'SNOW': 'SNOW_O'})\
.drop(columns=['DATE','airport'])

### 添加目的地天气状况
data_orig = pd.merge(data_orig, weather, how='left', left_on=['FlightDate','Dest'], right_on = ['DATE','airport'])\
.rename(columns = {'AWND':'AWND_D','PRCP':'PRCP_D', 'TAVG':'TAVG_D', 'SNOW': 'SNOW_D'})\
.drop(columns=['DATE','airport'])

**注意**：在合并数据后最好先检查空值/缺失值。

In [ ]:
sum(data.isna().any())

In [ ]:
data_orig.columns

使用独热编码将分类数据转化为数值数据。

In [ ]:
data = data_orig.copy()
data = data[['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
       'Reporting_Airline', 'Origin', 'Dest','Distance','DepHourofDay', 'is_delay', 'is_holiday', 'AWND_O', 'PRCP_O',
       'TAVG_O', 'AWND_D', 'PRCP_D', 'TAVG_D', 'SNOW_O', 'SNOW_D']]


categorical_columns = ['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
       'Reporting_Airline', 'Origin', 'Dest', 'is_holiday','is_delay']
for c in categorical_columns:
    data[c] = data[c].astype('category')

In [ ]:
data_dummies = # 在此处输入代码

### 示例代码

```
data_dummies = pd.get_dummies(data[['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'Reporting_Airline', 'Origin', 'Dest', 'is_holiday']], drop_first=True)
data = pd.concat([data, data_dummies], axis = 1)
categorical_columns.remove('is_delay')
data.drop(categorical_columns,axis=1, inplace=True)
```

检查新列。

In [ ]:
data.columns

### 示例输出

```
Index(['Distance', 'DepHourofDay', 'is_delay', 'AWND_O', 'PRCP_O', 'TAVG_O',
       'AWND_D', 'PRCP_D', 'TAVG_D', 'SNOW_O', 'SNOW_D', 'Year_2015',
       'Year_2016', 'Year_2017', 'Year_2018', 'Quarter_2', 'Quarter_3',
       'Quarter_4', 'Month_2', 'Month_3', 'Month_4', 'Month_5', 'Month_6',
       'Month_7', 'Month_8', 'Month_9', 'Month_10', 'Month_11', 'Month_12',
       'DayofMonth_2', 'DayofMonth_3', 'DayofMonth_4', 'DayofMonth_5',
       'DayofMonth_6', 'DayofMonth_7', 'DayofMonth_8', 'DayofMonth_9',
       'DayofMonth_10', 'DayofMonth_11', 'DayofMonth_12', 'DayofMonth_13',
       'DayofMonth_14', 'DayofMonth_15', 'DayofMonth_16', 'DayofMonth_17',
       'DayofMonth_18', 'DayofMonth_19', 'DayofMonth_20', 'DayofMonth_21',
       'DayofMonth_22', 'DayofMonth_23', 'DayofMonth_24', 'DayofMonth_25',
       'DayofMonth_26', 'DayofMonth_27', 'DayofMonth_28', 'DayofMonth_29',
       'DayofMonth_30', 'DayofMonth_31', 'DayOfWeek_2', 'DayOfWeek_3',
       'DayOfWeek_4', 'DayOfWeek_5', 'DayOfWeek_6', 'DayOfWeek_7',
       'Reporting_Airline_DL', 'Reporting_Airline_OO', 'Reporting_Airline_UA',
       'Reporting_Airline_WN', 'Origin_CLT', 'Origin_DEN', 'Origin_DFW',
       'Origin_IAH', 'Origin_LAX', 'Origin_ORD', 'Origin_PHX', 'Origin_SFO',
       'Dest_CLT', 'Dest_DEN', 'Dest_DFW', 'Dest_IAH', 'Dest_LAX', 'Dest_ORD',
       'Dest_PHX', 'Dest_SFO', 'is_holiday_1'],
      dtype='object')
```

再次将 `is_delay` 列重命名为 `target`。使用跟之前一样的代码。

In [ ]:
data.rename(columns = {<CODE>:<CODE>}, inplace=True )# 在此处输入代码

再次创建训练集。

In [ ]:
# 在此处输入代码

### 新基准分类器

现在我们来看一下这些新特征是否提高了模型的预测能力。

In [ ]:
# 实例化线性学习器评估程序对象
num_classes = # 在此处输入代码
classifier_estimator = # 在此处输入代码

### 示例代码

```
num_classes = len(pd.unique(train_labels)) 
classifier_estimator = sagemaker.LinearLearner(role=sagemaker.get_execution_role(),
                                               train_instance_count=1,
                                               train_instance_type='ml.m4.xlarge',
                                               predictor_type='binary_classifier',
                                              binary_classifier_model_selection_criteria = 'cross_entropy_loss')
```

In [ ]:
train_records = classifier_estimator.record_set(train_features, train_labels, channel='train')
val_records = classifier_estimator.record_set(val_features, val_labels, channel='validation')
test_records = classifier_estimator.record_set(test_features, test_labels, channel='test')

classifier_estimator.fit([train_records, val_records, test_records])

In [ ]:
classifier_predictor = # 在此处输入代码

In [ ]:
predict_batches(classifier_predictor, test_features, test_labels)

线性模型的效果只有少许提高。我们使用 Amazon SageMaker 来尝试基于树的集成模型 XGBoost。

### 尝试 XGBoost 模型

您需要采取的步骤如下：  

1. 使用训练集变量并将其另存为 CSV 文件 train.csv、validation.csv 和 test.csv  
2. 将存储桶名称存储在变量中。Amazon S3 存储桶名称位于实验说明左侧。 
a. `bucket = <LabBucketName>`  
b. `prefix = 'sagemaker/xgboost'`  
3. 使用 Boto3 将模型上传到存储桶。   

In [ ]:
train_data, validation_data, test_data = np.split(data.sample(frac=1, random_state=1729), [int(0.7 * len(data)), int(0.9*len(data))])  

pd.concat([train_data['target'], train_data.drop(['target'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
pd.concat([validation_data['target'], validation_data.drop(['target'], axis=1)], axis=1).to_csv('validation.csv', index=False, header=False)
pd.concat([test_data['target'], test_data.drop(['target'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)

用实验账户提供的资源名称替换 **`<LabBucketName>`**。

In [ ]:
# 在此处输入代码，以定义存储桶和 prefix
bucket = <LabBucketName> # 在此处输入代码
prefix = 'sagemaker/xgboost'

In [ ]:
### 将数据上传到 S3
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'xgboost')

使用 `sagemaker.s3_input` 函数为训练和验证数据集创建一个 record_set。

In [ ]:
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv')

In [ ]:
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(container,
                                    role = sagemaker.get_execution_role(),
                                    train_instance_count=1,
                                    train_instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        eval_metric = "auc",
                        num_round=100)

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})

针对新模型部署预测器并使用测试数据集来评估模型。

In [ ]:
xgb_predictor = xgb.deploy(initial_instance_count = <CODE>,
                           instance_type = <CODE>)# 在此处输入代码

In [ ]:
test_data = pd.concat([test_data['target'], test_data.drop(['target'], axis=1)], axis=1)

In [ ]:
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer
xgb_predictor.deserializer = None


def predict(predictor , features, labels , prob_threshold = 0.5, rows=500):
    """
    返回评估结果
    predictor：模型的预测器对象
    features：输入模型的特征
    label：Ground Truth 目标值
    prob_threshold：分隔阳性和阴性类的概率分界点
    """
    split_array = np.array_split(features, int(features.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, predictor.predict(array).decode('utf-8')])

    preds = np.fromstring(predictions[1:], sep=',')
    preds = preds.reshape((-1,))
    predictions = np.where(preds > prob_threshold , 1, 0)
    labels = labels.reshape((-1,))
    
    
    # 计算准确率
    accuracy = (predictions == labels).sum() / labels.shape[0]
    print(f'Accuracy: {accuracy}')
    
    auc = roc_auc_score(labels, preds)
    print(f'AUC : {auc}')
    
    precision, recall, f1_score, _ = precision_recall_fscore_support(labels, predictions, average = 'binary')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1_score: {f1_score}')
    
    
    confusion_matrix = pd.crosstab(index=labels, columns=np.round(preds), rownames=['True'], colnames=['predictions']).astype(int)
    plt.figure(figsize = (5,5))
    sns.heatmap(confusion_matrix, annot=True, fmt='.2f', cmap="YlGnBu").set_title('Confusion Matrix') 
    return list(preds)

In [ ]:
predictions = predict(xgb_predictor, test_data.as_matrix()[:, 1:] , test_data.iloc[:, 0:1].as_matrix(), prob_threshold = 0.5)

### 尝试不同阈值

In [ ]:
predictions = predict(xgb_predictor, test_data.as_matrix()[:, 1:] , test_data.iloc[:, 0:1].as_matrix(), prob_threshold = 0.8)

**问题**：您能从模型使用测试集的效果中推断出什么？

In [ ]:
# 在此处输入答案

### 超参数优化 (HPO)

In [ ]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

### 您可以启动多个实例同时进行超参数优化

xgb = sagemaker.estimator.Estimator(container,
                                    role=sagemaker.get_execution_role(),
                                    train_instance_count= 2, # 确保针对这些实例设置了限制
                                    train_instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)

xgb.set_hyperparameters(eval_metric='auc',
                        objective='binary:logistic',
                        num_round=100,
                        rate_drop=0.3,
                        tweedie_variance_power=1.4)

In [ ]:
hyperparameter_ranges = {'eta': ContinuousParameter(0, 0.5),
                        'min_child_weight': ContinuousParameter(3, 10),
                         'num_round': IntegerParameter(10, 150),
                        'alpha': ContinuousParameter(0, 2),
                        'max_depth': IntegerParameter(10, 15)}

objective_metric_name = 'validation:auc'

tuner = HyperparameterTuner(xgb,
                            objective_metric_name,
                            hyperparameter_ranges,
                            max_jobs=10,
                            max_parallel_jobs=3)

In [ ]:
tuner.fit({'train': s3_input_train, 'validation': s3_input_validation})

In [ ]:
boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']

<i class="fas fa-exclamation-triangle" style="color:red"></i> 等待训练作业完成。这可能需要 25 到 30 分钟的时间。

**要监控超参数优化作业，您需要进行以下操作：**  

1. 在 AWS 管理控制台的 **Services（服务）**菜单中，单击 **Amazon SageMaker**。 
1. 单击 **Training（训练）**> **Hyperparameter tuning jobs（超参数优化作业）**。 
1. 您可以查看每个超参数优化作业的状态、目标指标值和日志。 

In [ ]:
sage_client = boto3.Session().client('sagemaker')
tuning_job_name = tuner.latest_tuning_job.job_name

#运行以下单元，查看超参数优化作业的当前状态。
tuning_job_result = sage_client.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=tuning_job_name)

status = tuning_job_result['HyperParameterTuningJobStatus']
if status != 'Completed':
    print('Reminder: the tuning job has not been completed.')
    
job_count = tuning_job_result['TrainingJobStatusCounters']['Completed']
print("%d training jobs have completed" % job_count)
    
is_minimize = (tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['Type'] != 'Maximize')
objective_name = tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['MetricName']


from pprint import pprint
if tuning_job_result.get('BestTrainingJob',None):
    print("Best model found so far:")
    pprint(tuning_job_result['BestTrainingJob'])
else:
    print("No training jobs have reported results yet.")
    
### 获取 DataFrame 中的超参数优化作业结果
tuner_df = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name).dataframe()

if len(tuner_df) > 0:
    df = tuner_df[tuner_df['FinalObjectiveValue'] > -float('inf')]
    if len(df) > 0:
        df = df.sort_values('FinalObjectiveValue', ascending=is_minimize)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest":min(df['FinalObjectiveValue']),"highest": max(df['FinalObjectiveValue'])})
        pd.set_option('display.max_colwidth', -1) # 请勿截断 TrainingJobName        
    else:
        print("No training jobs have reported valid results yet.")
        
tuner_df

部署经过超参数优化训练的最佳模型。

In [ ]:
xgb_predictor_hpo = # 在此处输入代码。

In [ ]:
xgb_predictor_hpo.content_type = 'text/csv'
xgb_predictor_hpo.serializer = csv_serializer
xgb_predictor_hpo.deserializer = None

predictions = predict(xgb_predictor_hpo, test_data.as_matrix()[:, 1:] , test_data.iloc[:, 0:1].as_matrix(), prob_threshold = 0.5)

**问题**：尝试不同的超参数和超参数范围。这会改进模型吗？

### 特征重要性

所有超参数优化作业的模型文件都按上表中训练作业名称的顺序保存在“{bucket}/{prefix}/output/”文件夹中。您可以加载这些模型，并像使用常规 sklearn 模型对象一样对其进行解释。

In [ ]:
#best_hpo_model_path = "s3://" + bucket + "/sagemaker/xgboost/output/<Best model TrainingJobName>/output/model.tar.gz"
best_hpo_model_path = <path-to-your-model>
### 将 best_hpo_model_path 下载到本地
!aws s3 cp {best_hpo_model_path} .

In [ ]:
# 安装 xgboost
!pip install xgboost=='0.90'

#### 加载选择的模型文件

In [ ]:
import pickle
import tarfile
import xgboost

with open('model.tar.gz', 'rb') as f:
    with tarfile.open(fileobj=f, mode='r') as tar_f:
        with tar_f.extractfile('xgboost-model') as extracted_f:
            xgbooster = pickle.load(extracted_f)

#### 将列名映射到 XGBoost 模型

In [ ]:
columns = list(data.columns)
columns.remove('target')

In [ ]:
feature_importance = xgbooster.get_fscore()
feature_importance_col = {}

for column, fname in zip(columns, xgbooster.feature_names):
    try:
         feature_importance_col[column] = feature_importance[fname]
    except Exception:
        pass

#### 按特征重要性值进行排序

In [ ]:
sorted(feature_importance_col.items(), key=lambda kv: kv[1], reverse=True)

### 示例输出

```
[('AWND_O', 13851),
 ('AWND_D', 13452),
 ('DepHourofDay', 13344),
 ('TAVG_O', 13106),
 ('TAVG_D', 12800),
 ('Distance', 8478),
 ('PRCP_O', 4210),
 ('PRCP_D', 3916),
 ('Reporting_Airline_UA', 1791),
 ('Year_2016', 1290),
 ('Year_2015', 1285),
 ('Year_2018', 1157),
 ('Quarter_4', 1092),
 ('Year_2017', 1009),
 ('DayOfWeek_5', 838),
 ('DayOfWeek_4', 833),
 ('Quarter_2', 799),
 ('DayOfWeek_7', 783),
 ('Reporting_Airline_WN', 736),
 ('DayOfWeek_3', 718),
 ('Origin_ORD', 706),
 ('DayOfWeek_2', 685),
 ('Reporting_Airline_DL', 663),
 ('Dest_LAX', 627),
 ('DayOfWeek_6', 614),
 ('Reporting_Airline_OO', 596),
 ('Origin_LAX', 590),
 ('Month_11', 565),
 ('Month_5', 543),
 ('Dest_ORD', 539),
 ('SNOW_O', 506),
 ('Month_8', 497),
 ('Month_12', 495),
 ('Month_7', 484),
 ('Origin_DFW', 480),
 ('Quarter_3', 477),
 ('Dest_DFW', 462),
 ('Origin_DEN', 456),
 ('Origin_SFO', 438),
 ('Month_6', 437),
 ('Dest_SFO', 437),
 ('Month_10', 422),
 ('Month_9', 407),
 ('Month_4', 394),
 ('Dest_DEN', 379),
 ('SNOW_D', 375),
 ('Month_3', 369),
 ('Dest_IAH', 369),
 ('Origin_PHX', 347),
 ('Dest_PHX', 346),
 ('Origin_IAH', 342),
 ('DayofMonth_15', 324),
 ('DayofMonth_17', 322),
 ('DayofMonth_21', 322),
 ('Origin_CLT', 322),
 ('DayofMonth_20', 316),
 ('DayofMonth_9', 313),
 ('DayofMonth_19', 307),
 ('DayofMonth_26', 303),
 ('DayofMonth_7', 301),
 ('DayofMonth_4', 300),
 ('DayofMonth_2', 299),
 ('DayofMonth_27', 298),
 ('DayofMonth_3', 294),
 ('DayofMonth_28', 287),
 ('Month_2', 284),
 ('DayofMonth_5', 284),
 ('DayofMonth_12', 282),
 ('DayofMonth_11', 276),
 ('Dest_CLT', 276),
 ('DayofMonth_6', 275),
 ('DayofMonth_22', 274),
 ('DayofMonth_8', 273),
 ('DayofMonth_18', 273),
 ('DayofMonth_29', 267),
 ('DayofMonth_23', 266),
 ('DayofMonth_14', 263),
 ('DayofMonth_30', 260),
 ('DayofMonth_10', 259),
 ('DayofMonth_16', 257),
 ('DayofMonth_24', 255),
 ('DayofMonth_13', 245),
 ('DayofMonth_25', 237),
 ('is_holiday_1', 231),
 ('DayofMonth_31', 164)]
```

根据上述特征重要性，您可以发现，出发地和目的地的 DepHourofDay、Airwind 和 Temperature 是决定航班是否延误的主要影响因素。

通过这种方式，我们验证了在特征方面有哪些直观的信息，以及模型实际实际学习到了什么。

## 总结

现在，您已经至少完成了几次训练和评估模型的迭代。我们来总结一下该项目，回顾自己学到的内容，并思考可以采取哪些步骤来继续学习（假设您还有时间）。使用下面的单元格回答其中一些问题和其他相关问题：

1. 您的模型性能是否能实现您的业务目标？ 如果不符合，而且您还有时间进行优化，那么您会采取哪些不同的做法？
2. 在您对数据集、特征和超参数做出更改之后，模型有多大程度的改进？ 在整个项目中，您认为自己采用的哪些方法对模型的改进最大？
3. 在整个项目中，您遇到的最大挑战是什么？
4. 管道的某些方面对您来说没有意义，关于这些方面，您有没有未解决的问题？
5.在完成这个项目的过程中，关于机器学习，您学到的三件最重要的事情是什么？

#### <span style="color: blue;">项目演示：同样在项目演示中总结这些问题的答案。现在，请整理您为项目演示做下的所有笔记，并准备向全班展示您的成果。</span>